In [ ]:
import math
import logging
import numpy as np
import pandas as pd
import scipy.signal as sig
import altair as alt
import flammkuchen as fl

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.load import graph_melt
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq
from spinorama.compute_scores import scores
from spinorama.filter_scores import scores_apply_filter, scores_print

In [ ]:
df = fl.load('../cache.parse_all_speakers.h5')

In [ ]:
df_speaker = df['Adam S2V']['ASR']['asr']
df_speaker.keys()
original_mean = df_speaker['CEA2034_original_mean']

In [ ]:
g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -50, 'ymax': 10, 'width': 400, 'height': 250}
score = scores(df_speaker)
# print(score)
# print(score['pref_score'])

In [ ]:
my_fs = 48000
my_peq = parse_eq_iir_rews('../datas/eq/Adam S2V/iir.txt', my_fs)
spin_filtered, pir_filtered, score_filtered = scores_apply_filter(df_speaker, my_peq)
# print(score_filtered)

In [ ]:
scores_print(score, score_filtered)

In [ ]:
graph_spinorama(df_speaker['CEA2034'], g_params) | graph_spinorama(spin_filtered, g_params)